## Importing Data from CSV

In [3]:
import pandas as pd
import numpy as np

In [4]:
url = "https://raw.github.com/pandas-dev/pandas/main/pandas/tests/io/data/csv/tips.csv"

tips = pd.read_csv(url)
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


The `.head()` function gets you the first 5 elements of the data frame to show what the data would look like.

You could also use the function `describe()` which return summary statistics about the dataset

In [5]:
tips.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


### Downloading Dataset from Kaggle using `OpenDatasets`

In cases where you're trying to use datasets that are available on online sources like `Kaggle`. You can continue to use the URL directly, if the source makes one available. Or you could use a tool such as `opendatasets`.
`opendatasets` is a Python library for downloading datasets from online sources like `Kaggle` and `Google Drive` using a simple Python command.

The following examples show how you can (DOWNLOAD) the US Elections Dataset available via `Kaggle`. You will be asked to provide your username and authentication API key (that's not the same as your account password).

**To get your Kaggle API Key:**
![Where to get your Kaggle API Key](./assets/kaggle-profile.jpg)

In [8]:
import opendatasets as od
dataset_url = 'https://www.kaggle.com/tunguz/us-elections-dataset'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading us-elections-dataset.zip to ./us-elections-dataset


100%|██████████| 133k/133k [00:00<00:00, 1.49MB/s]

Once downloaded, you can Import the CSV file into a data frame.

In [4]:
## import CSv file from local folder into a dataframe
elections = pd.read_csv('./us-elections-dataset/1976-2020-president.csv')
elections.head()


,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,504070,1182850,20210113,NaN,REPUBLICAN
2,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,False,9198,1182850,20210113,NaN,OTHER
3,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,False,6669,1182850,20210113,NaN,OTHER
4,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"HALL, GUS",COMMUNIST PARTY USE,False,1954,1182850,20210113,NaN,OTHER


## Import Data from an API
To import data using an HTTP API in Python, you have multiple options for the HTTP client libraries. The one I choose and use here is `requests`. Here I will show you how you can use `requests` to query data from the US Census Data, and from the John Hopkins COVID APIs.

The most basic example is the following:
```python
import requests

x = requests.get('https://w3schools.com/python/demopage.htm')

print(x.text)
```

However, depending on the API provider, we may need to pass additional configurations and options in the request. We'll see that in the following examples

### US Census Data
The US Census Bureau provides machine-readable dataset via a developer API. 

> Developers could use the statistics available through this API to create apps that:
> 1. Show commuting patterns for every city in America.
> 2. Display the latest numbers on owners and renters in a neighborhood someone may want to live in.
> 3. Provide a local government a range of socioeconomic statistics on its population.

Here are [some of the datasets available that you can use](https://www.census.gov/data/developers/data-sets.html)

Each dataset provides technical documentation for the different variables you could get from that Particular API. Here's [an example](https://api.census.gov/data/2020/dec/pl/variables.html)


- [Working with US Census Data - PDF Guide](https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_api_handbook_2020_ch02.pdf)

To construct a request with `requests` we need to determine the URL we need to send the request to.

In [10]:
import requests

HOST = "https://api.census.gov/data"
year = "2022"
dataset = "cps/basic/apr"
base_url = "/".join([HOST, year, dataset]) # JOIN the variables with a `/` separator https://api.census.gov/data/2022/cps/basic/apr

# The dataset is huge and contains a lot of data, so we'll request a subset of the available variables.
dataset_variables = ["GEDIV","HRMIS","PENATVTY"] 

predicates = {}
predicates["get"] = ",".join(dataset_variables) # JOIN the variables with a `,` separator
predicates["for"] = "state:*"

response = requests.get(base_url, params=predicates)

census_data = pd.DataFrame.from_records(response.json()[1:], columns=response.json()[0])
print(census_data.head())

  GEDIV HRMIS PENATVTY state
0     8     8      303     4
1     8     8      303     4
2     8     8       57     4
3     8     8       57     4
4     7     2       57     5


This [link here](https://api.census.gov/data/2022/cps/basic/apr/variables.html) describes what those variables mean.

Also, you note that even the states are presented with some numerical values. to get the values of those state IDs, I'll send another request to another dataset

In [12]:
import requests

HOST = "https://api.census.gov/data"
year = "2017"
dataset = "acs/acs5"
base_url = "/".join([HOST, year, dataset]) # JOIN the variables with a `/` separator https://api.census.gov/data/2022/cps/basic/apr

dataset_variables = ["NAME"]
predicates = {}
predicates["get"] = ",".join(dataset_variables)
predicates["for"] = "state:*"

r = requests.get(base_url, params=predicates)
states = pd.DataFrame.from_records(r.json()[1:], columns=r.json()[0])
print(states.head())

          NAME state
0  Mississippi    28
1     Missouri    29
2      Montana    30
3     Nebraska    31
4       Nevada    32


You could subset the data recieved to find the State Numerical Code for the `Ohio`

In [13]:
# Find the state code with the name "Ohio"
print(states[states["NAME"] == "Ohio"])

    NAME state
16  Ohio    39


or you could just combine and merge the 2 data sets on the state ID, so we end up with a single dataset with all the information we need.

In [14]:
# Merge census_data and states
full_census_data = pd.merge(census_data, states, on="state")
full_census_data.head() # Print the first 5 rows

,GEDIV,HRMIS,PENATVTY,state,NAME
0,7,6,57,40,Oklahoma
1,7,6,57,40,Oklahoma
2,7,6,57,40,Oklahoma
3,7,6,57,40,Oklahoma
4,7,3,57,40,Oklahoma


### John Hopkins Covid API

John Hopkins University | Cornoavirus Resources Center have been collecting and publishing data regarding COVID-19 since day one. 
You can learn more about that and [find official resources for how to use the data here](https://coronavirus.jhu.edu/about/how-to-use-our-data)

I believe someone is publishing those datasets by means of API through the RapidAPI platform. Whether it's an official resource or not, the goal of this exercise is to show you how to work with APIs in general.

You can see a very easy to understand documentation of [how to use the API here](https://rapidapi.com/axisbits-axisbits-default/api/covid-19-statistics/). 


In [56]:
import requests

url = "https://covid-19-statistics.p.rapidapi.com/reports"

querystring = {"region_province":"Ohio","iso":"USA","region_name":"US","q":"US Ohio","date":"2020-04-16"}

headers = {
	"X-RapidAPI-Key": "b6d38dbbd1msh33a9b59e4f6ddefp148a30jsn8bb7487ef097",
	"X-RapidAPI-Host": "covid-19-statistics.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.json()["data"])

# save json result into a pandas dataframe
covid_data = pd.DataFrame(response.json()["data"])
covid_data.head()

[{'date': '2020-04-16', 'confirmed': 8414, 'deaths': 407, 'recovered': 0, 'confirmed_diff': 620, 'deaths_diff': 45, 'recovered_diff': 0, 'last_update': '2020-04-16 23:30:51', 'active': 8007, 'active_diff': 575, 'fatality_rate': 0.0484, 'region': {'iso': 'USA', 'name': 'US', 'province': 'Ohio', 'lat': '40.3888', 'long': '-82.7649', 'cities': [{'name': 'Adams', 'date': '2020-04-16', 'fips': 39001, 'lat': '38.84541072', 'long': '-83.4718964', 'confirmed': 3, 'deaths': 0, 'confirmed_diff': 0, 'deaths_diff': 0, 'last_update': '2020-04-16 23:30:51'}, {'name': 'Allen', 'date': '2020-04-16', 'fips': 39003, 'lat': '40.77285242', 'long': '-84.10802343', 'confirmed': 57, 'deaths': 6, 'confirmed_diff': 6, 'deaths_diff': 2, 'last_update': '2020-04-16 23:30:51'}, {'name': 'Ashland', 'date': '2020-04-16', 'fips': 39005, 'lat': '40.84772277', 'long': '-82.27280781', 'confirmed': 5, 'deaths': 0, 'confirmed_diff': -1, 'deaths_diff': 0, 'last_update': '2020-04-16 23:30:51'}, {'name': 'Ashtabula', 'date':

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,region
0,2020-04-16,8414,407,0,620,45,0,2020-04-16 23:30:51,8007,575,0.0484,"{'iso': 'USA', 'name': 'US', 'province': 'Ohio..."


## Importing Data using Socrata
The Socrata Open Data API allows you to programmatically access a wealth of open data resources from governments, non-profits, and NGOs around the world. 

### Cincinnati Datasets
For example, [here are a bunch of datasets that you can find about the City of Cincinnati](https://www.opendatanetwork.com/entity/1600000US3915000/Cincinnati_OH/demographics.population.count?ref=search-entity&year=2018)

For you to be able to access any of those datasets, you need to register to get an application token.

![Cincinnati Data Portal](./assets/cin.jpg)


<!-- app_token = 'CykQ9zQKgUoasyKaHdVYptQQ3' -->

In [58]:
import pandas as pd
from sodapy import Socrata

# Example authenticated client (needed for non-public datasets):
app_token = 'REDACTED'

# client = Socrata("data.cincinnati-oh.gov", None)
client = Socrata("data.cincinnati-oh.gov",app_token)


# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("rvmt-pkmq", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
print(results_df.columns)

Index(['address_x', 'latitude_x', 'longitude_x',
       'community_council_neighborhood', 'cpd_neighborhood',
       'sna_neighborhood', 'crashdate', 'crashseverity', 'crashseverityid',
       'datecrashreported', 'dayofweek', 'injuries', 'instanceid',
       'lightconditionsprimary', 'localreportno', 'mannerofcrash',
       'roadconditionsprimary', 'roadcontour', 'roadsurface', 'unittype',
       'typeofperson', 'weather', 'zip', 'age', 'gender', 'roadclass',
       'roadclassdesc'],
      dtype='object')


## Importing Data from SQL (Work-in-progress)

![DB Schema](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)


In [60]:
from sqlalchemy import create_engine

# engine = create_engine("sqlite:///:memory:")
engine = create_engine("sqlite:///chinook.db")

dbConnection = engine.connect()

In [64]:
df = pd.read_sql("SELECT * FROM genres;", dbConnection)
df

,GenreId,Name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll
5,6,Blues
6,7,Latin
7,8,Reggae
8,9,Pop
9,10,Soundtrack


<!-- # ---
# sidebar_position: 2
# title: SQL
# draft: true
# ---
# - [ ] the differences between MSSQL, MySQL, PGSQL.
#   - [ ] how would you choose one over the other?
#   - [ ] Does Python have an ORM that abstracts away that?
# - [ ] That would be a research question for them.

# ## PostgreSQL
# - Partitioned Table
#   - What is a pg partitioned table
# - [ ] I will need to provide some virtual labs for the student such that they can ssh into the server from their computer

# ### Ideas for rubrics
# - What were the sources? are they recent? are they credible?
# - What versions were they comparing

# Adventure works
# https://github.com/chriseaton/docker-adventureworks/blob/master/mssql/build.sh


# Automate grading sql
# https://education.github.community/t/autograding-sql-statements-for-mysql/63451/2
# https://github.com/markpatterson27/Simple-SQL-Exercise

# - https://github.com/devrimgunduz/pagila
# - https://github.com/Jkremr/pagila-queries -->